In [11]:
import json
import networkx as nx
from networkx.algorithms.simple_paths import shortest_simple_paths


with open("cable-and-landing-points.json", "r") as cable_and_landing_points_file:
    cable_and_landing_points = json.load(cable_and_landing_points_file)

In [12]:
allowed_countries = {
    "India", "Australia", "Sri Lanka", "Myanmar", "Thailand", "Cambodia",
    "Vietnam", "Malaysia", "Indonesia", "Singapore", "Papua New Guinea",
    "Brunei", "Timor-Leste", "Philippines", "Solomon Islands"
}

def normalize_node(landing_point):
    return f"{landing_point['city'].strip()}, {landing_point['country'].strip()}"

def get_coords(landing_point):
    return tuple(landing_point["coordinates"])

G = nx.Graph()
node_coords = {} 
india_nodes = set()
australia_nodes = set()
allowed_nodes = set()

for cable in cable_and_landing_points:
    points = cable["landing_points"]
    
    for i in range(len(points)):
        node_i = normalize_node(points[i])
        coord_i = get_coords(points[i])
        country_i = points[i]['country']

        node_coords[node_i] = coord_i

        if country_i == "India":
            india_nodes.add(node_i)
        elif country_i == "Australia":
            australia_nodes.add(node_i)
        if country_i in allowed_countries:
            allowed_nodes.add(node_i)

        for j in range(i + 1, len(points)):
            node_j = normalize_node(points[j])
            coord_j = get_coords(points[j])
            country_j = points[j]['country']

            node_coords[node_j] = coord_j
            if country_j in allowed_countries:
                allowed_nodes.add(node_j)

            # Only add edge if it's between different countries
            if country_i != country_j:
                G.add_edge(node_i, node_j, cable=cable["cable_name"])


subG = G.subgraph(allowed_nodes)

def is_valid_path(path):
    return all(node in allowed_nodes for node in path)

MAX_PATHS_PER_PAIR = 10
all_valid_paths = []

# Keep only nodes that are present in the subgraph
india_nodes = india_nodes & set(subG.nodes)
australia_nodes = australia_nodes & set(subG.nodes)


for source in india_nodes:
    for target in australia_nodes:
        try:
            path_gen = shortest_simple_paths(subG, source, target)
            count = 0
            for path in path_gen:
                if count >= MAX_PATHS_PER_PAIR:
                    break
                if is_valid_path(path):
        
                    cables = []
                    for i in range(len(path) - 1):
                        edge_data = subG.get_edge_data(path[i], path[i+1])
                        if edge_data:
                            cables.append(edge_data.get("cable"))
                    all_valid_paths.append({
                        "path": path,
                        "cables": cables
                    })
                    count += 1
        except nx.NetworkXNoPath:
            continue

geojson_features = []

for item in all_valid_paths:
    path = item["path"]
    cables = item["cables"]
    coordinates = [node_coords[node] for node in path]

    feature = {
        "type": "Feature",
        "properties": {
            "path": path,
            "cables": cables
        },
        "geometry": {
            "type": "LineString",
            "coordinates": coordinates
        }
    }
    geojson_features.append(feature)

geojson_output = {
    "type": "FeatureCollection",
    "features": geojson_features
}

with open("india_australia_paths.geojson", "w") as f:
    json.dump(geojson_output, f, indent=2)
no_domestic_hops_features = []

for item in all_valid_paths:
    path = item["path"]
    cables = item["cables"]
    coordinates = [node_coords[node] for node in path]

    # Initialize fault counts (replace with actual fault data if available)
    fault_counts = {cable: 0 for cable in cables}

    feature = {
        "type": "Feature",
        "properties": {
            "path": path,
            "cables": cables,
            "cable_count": len(cables),
            "fault_counts": fault_counts
        },
        "geometry": {
            "type": "LineString",
            "coordinates": coordinates
        }
    }

    no_domestic_hops_features.append(feature)

# Wrap in a FeatureCollection
no_domestic_hops_geojson = {
    "type": "FeatureCollection",
    "features": no_domestic_hops_features
}

# Save to file
with open("no_domestic_hops.json", "w") as f:
    json.dump(no_domestic_hops_geojson, f, indent=2)

for item in all_valid_paths:
    print(" → ".join(item["path"]))
    print("  Cables:", " → ".join(item["cables"]))
    print()


Kochi, India → Jakarta, Indonesia → Sydney, Australia
  Cables: ICE IV → Hawaiki Nui 1

Kochi, India → Singapore, Singapore → Sydney, Australia
  Cables: ICE IV → Hawaiki Nui 1

Kochi, India → Ancol, Indonesia → Sydney, Australia
  Cables: ICE IV → Hawaiki Nui 1

Kochi, India → Jakarta, Indonesia → Port Moresby, Papua New Guinea → Sydney, Australia
  Cables: ICE IV → Hawaiki Nui 1 → Coral Sea Cable System (CS²)

Kochi, India → Singapore, Singapore → Batam, Indonesia → Sydney, Australia
  Cables: ICE IV → Hawaiki Nui 1 → Hawaiki Nui 1

Kochi, India → Ancol, Indonesia → Port Moresby, Papua New Guinea → Sydney, Australia
  Cables: ICE IV → Hawaiki Nui 1 → Coral Sea Cable System (CS²)

Kochi, India → Jakarta, Indonesia → Honiara, Solomon Islands → Sydney, Australia
  Cables: ICE IV → Hawaiki Nui 1 → Coral Sea Cable System (CS²)

Kochi, India → Singapore, Singapore → Port Moresby, Papua New Guinea → Sydney, Australia
  Cables: ICE IV → Hawaiki Nui 1 → Coral Sea Cable System (CS²)

Kochi, In

In [13]:
# node_coords = {} 
# india_nodes = set()
# australia_nodes = set()
print(allowed_nodes)

{'Natuna, Indonesia', 'Batangas, Philippines', 'Bintulu, Malaysia', 'Naga, Philippines', 'Leganes, Philippines', 'Sangatta, Indonesia', 'Wewak, Papua New Guinea', 'Surigao City, Philippines', 'Singapore, Singapore', 'Lingga, Indonesia', 'Ketapang, Indonesia', 'Kawinda Nae, Indonesia', 'Seraya, Indonesia', 'Padang, Indonesia', 'Inverloch, Australia', 'Thanlyin, Myanmar', 'San Jose, Philippines', 'Agats, Indonesia', 'Roxas City, Philippines', 'Taytay, Philippines', 'Masbate City, Philippines', 'Kalpeni, India', 'Quy Nhon, Vietnam', 'Santa Magdalena, Philippines', 'Liloy, Philippines', 'Dili, Timor-Leste', 'Raha, Indonesia', 'Bulan, Philippines', 'Ambon, Indonesia', 'Kampung Raja, Malaysia', 'Claveria, Philippines', 'Kuala Sedili, Malaysia', 'Tanjung Bemban, Indonesia', 'Butuan City, Philippines', 'Ancol, Indonesia', 'Coron, Philippines', 'Saraemee, Indonesia', 'Popondetta, Papua New Guinea', 'Tanjung Selor, Indonesia', 'Kavieng, Papua New Guinea', 'Buranga, Indonesia', 'Alyangula, Austra

In [ ]:
print(len(all_valid_paths))

300
Note: you may need to restart the kernel to use updated packages.


In [15]:
import folium

with open("india_australia_paths.geojson") as f:
    geojson_data = json.load(f)

m = folium.Map(location=[-10, 100], zoom_start=4, tiles="cartodb positron")

for feature in geojson_data["features"]:
    coords = feature["geometry"]["coordinates"]
    cables = feature["properties"]["cables"]
    path = feature["properties"]["path"]

    folium.PolyLine(
        locations=[(lat, lon) for lon, lat in coords], 
        color="blue",
        weight=3,
        opacity=0.7,
        tooltip=" → ".join(cables)
    ).add_to(m)

    start = coords[0]
    end = coords[-1]
    folium.Marker(
        location=(start[1], start[0]),
        popup=f"Start: {path[0]}",
        icon=folium.Icon(color="green", icon="play")
    ).add_to(m)
    folium.Marker(
        location=(end[1], end[0]),
        popup=f"End: {path[-1]}",
        icon=folium.Icon(color="red", icon="stop")
    ).add_to(m)

m.save("india_australia_paths_map.html")
# print("Map saved to india_australia_paths_map.html")
